In [ ]:
import requests
import time
import random
import re
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#Initiate all lists to store scraped data
song_url_list = [] 
song_artist_list = []
song_title_list = []
song_year_list = []
song_lyrics_list = []

In [ ]:
#get response for "w" artists page
letter_url = "https://www.azlyrics.com/w.html"
letter_page = requests.get(letter_url)
letter_soup = BeautifulSoup(letter_page.content, 'lxml')
print("status:", letter_page.status_code)

In [ ]:
#scrape letter "w" page for artists
artist_columns = letter_soup.find_all("div", class_="col-sm-6 text-center artist-col")
left = artist_columns[0]
right = artist_columns[1]
    
left_artists = left.find_all("a")
right_artists = right.find_all("a")
artist_list = left_artists + right_artists

print(len(artist_list))

In [ ]:
#initialize count to iterate
count = 0

In [ ]:
# Scrape to get populate lists with songs in "W"
while count < len(artist_list):
    artist_link = artist_list[count].get("href")
    #artist_link = artist_list[0].get("href")
    artist_url = "https://www.azlyrics.com/" + artist_link
    artist_page = requests.get(artist_url)
    
    print(count, artist_url)
    print("status:", artist_page.status_code)
    
    artist_soup = BeautifulSoup(artist_page.content, 'lxml')

    # artist page
    artist_name = artist_soup.find("h1").findChild("strong").text
    all_songs = artist_soup.find_all("div", class_="listalbum-item")

    for song in all_songs:
        #song_artist
        #print(artist_name[:-7])
        song_artist_list.append(artist_name[:-7])
    
    
        #song_title
        song_title = song.find('a').text
        #print(song_title)
        song_title_list.append(song_title)
        
        
        #song_year
        song_year = song.find_previous_sibling("div", class_="album")
        if song_year:
            year = song_year.text
            if re.search(r"\(\d+\)", year):
                match = re.search(r"\((\d+)\)", year)
                #print(match.group(1))
                song_year_list.append(match.group(1))
            else: 
                #print("")
                song_year_list.append("")
        else:
            #print("")
            song_year_list.append("")
    
    
        #song_url
        song_link = song.find('a').get('href')
        if "https://" in song_link:
            #print(song_link)
            song_url_list.append(song_link)
        else:
            song_link_edit = "https://www.azlyrics.com/" + song_link[3:]
            #print(song_link_edit)
            song_url_list.append(song_link_edit)
    
    
    print(len(song_artist_list), "|", len(song_title_list), "|", len(song_year_list), "|", len(song_url_list))
    
    count += 1
    time.sleep(random.randint(7,28))
        
   
print("done")

In [ ]:
#create new dataframe with values from lists of data
data = {'artist_name': song_artist_list, 'song_title': song_title_list, 'year': song_year_list, 'lyrics_url': song_url_list}
df = pd.DataFrame(data, columns= ['artist_name', 'song_title','year','lyrics_url'])

df['year'] = df['year'].apply(pd.to_numeric)

df.info()
df.tail()

#save all data to csv
df.to_csv('arthurwu_project1_allSongs_urls.csv', index = False, header=True)


In [ ]:
#new dataframe for sampled songs
sample_df = pd.DataFrame()

# get up to 10 samples from every year
for i in range(int(df["year"].min()), int(df["year"].max() + 1)):
    year_songs = df[df["year"] == i]
    if len(year_songs) < 10 and len(year_songs) > 0:
        sample_df = pd.concat([sample_df, year_songs], axis=0, ignore_index=True)
    elif len(year_songs) > 10:   
        year_sample = year_songs.sample(10)
        sample_df = pd.concat([sample_df, year_sample], axis=0, ignore_index=True)
    else:
        print("no songs", i)
    
sample_df.info()
sample_df.tail(15)


In [ ]:
#initialize count to iterate
url_count = 0

In [ ]:
# get lyrics using sample url list
while url_count < len(sample_df["lyrics_url"]):
    
    #url request
    song_page = requests.get(sample_df["lyrics_url"][url_count])
    #song_page = requests.get(song_url)
    song_soup = BeautifulSoup(song_page.content, 'lxml')
    print(url_count, sample_df["lyrics_url"][url_count])
    
    # find and save lyrics data
    song_lyrics = song_soup.find_all("div", class_=None)[1].text.replace(",", ";")
    song_lyrics_list.append(song_lyrics)
    
    
    url_count += 1
    time.sleep(random.randint(7,28))

print("done")


In [ ]:
#add lyrics array to sample dataframe
sample_df["lyrics_text"] = song_lyrics_list    # sample_df and song_lyrics_list must match in len
sample_df.info()
sample_df.head()

#save sample dataframe as csv
sample_df1.to_csv('arthurwu_project1_sampledSongs_urls.csv', index = False, header=True)

